In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.search.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

#test 

import os
import googleapiclient.discovery

BASE_DIR = '../scraping'
KEY_PATH = os.path.join(BASE_DIR, "api_keyPG.txt")

with open(KEY_PATH) as f:
    api_key = f.readline()


In [20]:
def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.search().list(
        part="snippet",
        maxResults=50,
        pageToken="CDIQAA",
        q="surfing",
        type="video",
        videoCategoryId="26"
    )
    response = request.execute()

    return response

soup_ = main()['items']

In [22]:
soup_[0]['id']['videoId']


'anzPk-m2Ek8'

In [ ]:
print(main())

In [26]:
def get_vid_ids(dict_list):
    key = soup_[0]['id']['videoId']
    ids = []

    for i in range(len(dict_list)):
        soup_[0]['id']['videoId']
    return ids


In [24]:

print(get_vid_ids)


<function get_vid_ids at 0x10bab3370>


In [ ]:

if __name__ == "__main__":
    main()